<a href="https://colab.research.google.com/github/damyl4sure/google-collab/blob/main/twitterApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy

from twitter import *
import time

from dotenv import load_dotenv
load_dotenv()

import os


## Environment variable pass from .env file
ENV_MY_BEARER_TOKEN = os.getenv("MY_BEARER_TOKEN")

ENV_CONSUMER_KEY = os.getenv("CONSUMER_KEY")
ENV_CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")

ENV_ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
ENV_ACCESS_TOKEN_SECRET = os.getenv("ACCESS_TOKEN_SECRET")

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(ENV_CONSUMER_KEY, ENV_CONSUMER_SECRET)
 
# set access to user's access key and access secret
auth.set_access_token(ENV_ACCESS_TOKEN, ENV_ACCESS_TOKEN_SECRET)


## Authenticate to twitter API with credentials
client = tweepy.Client(
    bearer_token=ENV_MY_BEARER_TOKEN,
    consumer_key=ENV_CONSUMER_KEY,
    consumer_secret=ENV_CONSUMER_SECRET,
    access_token=ENV_ACCESS_TOKEN,
    access_token_secret=ENV_ACCESS_TOKEN_SECRET,
    wait_on_rate_limit=True
)

    
hoax_tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = '(#covid19 OR #virus OR #coronavirus) lang:en -is:retweet (place_country:US "and" (place:Florida OR place:"Panama city" OR place:Tallahassee OR place:Jacksonville OR place:Georgia OR place:Alabama))',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2020-01-01T00:00:00Z',
                                 end_time = '2021-01-31T00:01:00Z',
                              max_results=500):
    time.sleep(1)
    hoax_tweets.append(response)


hoax_tweets[0].data[0]

hoax_tweets[0].includes['users'][2]

hoax_tweets[0].data[0]

hoax_tweets[0].includes['users'][3]

hoax_tweets[0].includes['users'][3].location


import pandas as pd

tweet_info_ls = []
user_dict = {}

# timeout = 3600  # [seconds]
# timeout_start = time.time()

#while time.time() < timeout_start + timeout:
    
# Loop through each response object
for response in hoax_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    # for user in response.includes['users']:
    for tweet, user in zip(response.data, response.includes['users']):
        api = tweepy.API(auth, wait_on_rate_limit=True)
        #api = tweepy.API(auth)
        place = api.geo_id(tweet.geo['place_id'])
        tweet_info = {
            'tweet_id': tweet.id,
            'date': tweet.created_at,
            'geo': tweet.geo['place_id'],
            'bounding_box': str(place.bounding_box.coordinates),
            'bb_centroid': str(place.centroid),
            'coordinates': tweet.geo.get('coordinates'),
            'location': user.location,
            }
    tweet_info_ls.append(tweet_info)

tweets_df = pd.DataFrame(tweet_info_ls)

pd.set_option('display.max_rows', None)

#Remove non Florida tweets
tweets_df = tweets_df[tweets_df['location'].str.contains("Fl|FL|Florida|FLORIDA|Al|AL|Alabama|ALABAMA|Ga|GA|Georgia|GEORGIA", na = False)].reset_index(drop=True)

tweets_df['date'] = tweets_df['date'].dt.strftime('%m-%d-%Y')

blankIndex=[''] * len(tweets_df)
tweets_df.index=blankIndex

tweets_df.to_excel("output-data-from-twitter-latest-ap.xlsx")

print('DataFrame is written to Excel File successfully.')
